# Solving Nonlinear Boundary Value Problems

This notebook demonstrates solving nonlinear ordinary differential equations (ODEs) with boundary conditions using spectral methods. We'll explore:

1. Newton's method for nonlinear systems
2. The ultraspherical spectral method
3. Matrix-based Newton iteration
4. INGU: Inexact Newton-GMRES with FFT-based operators

The approach is based on **Qin & Xu (2024)** "Solving Nonlinear ODEs with the Ultraspherical Spectral Method" (IMA J. Numer. Anal.).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from funpy import Fun
from funpy.colloc.chebOp import ChebOp

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

## 1. Introduction to Nonlinear BVPs

A nonlinear boundary value problem (BVP) has the general form:

$$N[u](x) = 0, \quad x \in [a, b]$$
$$B[u] = 0 \quad \text{(boundary conditions)}$$

where $N$ is a nonlinear differential operator. Unlike linear problems, we cannot directly solve a matrix equation. Instead, we use **Newton's method** to iteratively find the solution.

## 2. Newton's Method for Nonlinear Systems

Given an approximate solution $u_k$, Newton's method computes the correction $\delta u$ by linearizing:

$$J[u_k] \, \delta u = -N[u_k]$$

where $J[u_k]$ is the **Jacobian** (Fr\'echet derivative) of $N$ at $u_k$. The update is:

$$u_{k+1} = u_k + \delta u$$

For the ODE $u'' + f(u) = 0$, the Jacobian is:

$$J[u_k] = \frac{d^2}{dx^2} + f'(u_k(x))$$

This is a **linear** operator with **variable coefficients** that depend on the current approximation $u_k$.

## 3. Example 1: A Simple Quadratic Nonlinearity

Let's solve the BVP:

$$u'' + u^2 = 1, \quad u(-1) = u(1) = 0$$

This has a smooth solution with $u(x) \approx -0.43$ at $x=0$.

In [ ]:
# Define the nonlinear BVP
op = ChebOp(functions=['u'], n=32, domain=[-1, 1])
op.eqn = ['diff(u, x, 2) + u**2 - 1']
op.bcs = ['u(-1)', 'u(1)']

# Solve using Newton's method
soln, success, res = op.solve(verbose=True)

print(f"\nConverged: {success}")
print(f"Residual: {res:.2e}")
print(f"Solution at x=0: {soln(0):.6f}")

In [ ]:
# Plot the solution
x = np.linspace(-1, 1, 200)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Solution
axes[0].plot(x, soln(x), 'b-', lw=2)
axes[0].set_xlabel('x')
axes[0].set_ylabel('u(x)')
axes[0].set_title(r"Solution to $u'' + u^2 = 1$")
axes[0].axhline(0, color='gray', linestyle='--', alpha=0.5)

# Verify: compute u'' + u^2 - 1 (should be ~0)
u_coeffs = soln.coeffs
residual_func = soln.diff(2) + soln**2 - 1
axes[1].semilogy(x, np.abs(residual_func(x)) + 1e-16, 'r-', lw=2)
axes[1].set_xlabel('x')
axes[1].set_ylabel(r"|$u'' + u^2 - 1$|")
axes[1].set_title('Residual (should be near machine precision)')

plt.tight_layout()
plt.show()

## 4. Example 2: The Bratu Equation

The **Bratu equation** is a classic nonlinear BVP:

$$u'' + \lambda e^u = 0, \quad u(-1) = u(1) = 0$$

For $\lambda < \lambda_c \approx 3.51$, there are two solutions. For $\lambda > \lambda_c$, no solution exists.

This problem arises in combustion theory and models thermal ignition.

In [ ]:
# Solve Bratu equation for different lambda values
lambdas = [0.5, 1.0, 2.0, 3.0]
solutions = []

for lam in lambdas:
    op = ChebOp(functions=['u'], n=48, domain=[-1, 1])
    op.eqn = [f'diff(u, x, 2) + {lam}*exp(u)']
    op.bcs = ['u(-1)', 'u(1)']
    
    soln, success, res = op.solve(verbose=False)
    if success:
        solutions.append((lam, soln))
        print(f"lambda = {lam}: u(0) = {soln(0):.6f}, residual = {res:.2e}")
    else:
        print(f"lambda = {lam}: Failed to converge")

In [ ]:
# Plot solutions
x = np.linspace(-1, 1, 200)

plt.figure(figsize=(10, 5))
for lam, soln in solutions:
    plt.plot(x, soln(x), lw=2, label=rf'$\lambda = {lam}$')

plt.xlabel('x', fontsize=12)
plt.ylabel('u(x)', fontsize=12)
plt.title(r"Bratu Equation: $u'' + \lambda e^u = 0$", fontsize=14)
plt.legend()
plt.axhline(0, color='gray', linestyle='--', alpha=0.5)
plt.show()

## 5. Example 3: Fisher's Equation (Bistable)

The **Fisher equation** with bistable nonlinearity:

$$u'' + r \cdot u(1 - u) = 0, \quad u(-1) = 1, \quad u(1) = 0$$

This models population dynamics with two stable states (0 and 1). The solution is a traveling front connecting these states.

In [ ]:
# Fisher equation with different reaction rates
rates = [4, 16, 64]
fisher_solutions = []

for r in rates:
    op = ChebOp(functions=['u'], n=64, domain=[-1, 1])
    op.eqn = [f'diff(u, x, 2) + {r}*u*(1 - u)']
    op.bcs = ['u(-1) - 1', 'u(1)']
    
    soln, success, res = op.solve(verbose=False)
    if success:
        fisher_solutions.append((r, soln))
        print(f"r = {r}: residual = {res:.2e}")

In [ ]:
# Plot Fisher solutions - traveling fronts
x = np.linspace(-1, 1, 200)

plt.figure(figsize=(10, 5))
for r, soln in fisher_solutions:
    plt.plot(x, soln(x), lw=2, label=f'r = {r}')

plt.xlabel('x', fontsize=12)
plt.ylabel('u(x)', fontsize=12)
plt.title(r"Fisher's Equation: $u'' + r \cdot u(1-u) = 0$", fontsize=14)
plt.legend()
plt.axhline(0, color='gray', linestyle='--', alpha=0.3)
plt.axhline(1, color='gray', linestyle='--', alpha=0.3)
plt.show()

print("\nNote: Larger r creates sharper fronts (steeper transitions).")

## 6. Example 4: Allen-Cahn Equation

The **Allen-Cahn equation** is a phase-field model:

$$\varepsilon u'' + u - u^3 = f(x), \quad u(-1) = 1, \quad u(1) = -1$$

The small parameter $\varepsilon$ creates a **singular perturbation** problem with internal layers. This is a challenging test for numerical methods.

In [ ]:
# Allen-Cahn with different epsilon values
epsilons = [0.2, 0.1, 0.05]
ac_solutions = []

for eps in epsilons:
    # Need more points for smaller epsilon (sharper layers)
    n = int(80 / eps**0.5)
    n = min(n, 256)  # Cap at reasonable size
    
    op = ChebOp(functions=['u'], n=n, domain=[-1, 1])
    # f(x) = sin(5x + 5) gives interesting structure
    op.eqn = [f'{eps}*diff(u, x, 2) + u - u**3 - sin(5*x + 5)']
    op.bcs = ['u(-1) - 1', 'u(1) + 1']
    
    soln, success, res = op.solve(verbose=False)
    if success:
        ac_solutions.append((eps, soln))
        print(f"eps = {eps}: n = {n}, residual = {res:.2e}")

In [ ]:
# Plot Allen-Cahn solutions
x = np.linspace(-1, 1, 500)

plt.figure(figsize=(10, 5))
for eps, soln in ac_solutions:
    plt.plot(x, soln(x), lw=2, label=rf'$\varepsilon = {eps}$')

plt.xlabel('x', fontsize=12)
plt.ylabel('u(x)', fontsize=12)
plt.title(r"Allen-Cahn: $\varepsilon u'' + u - u^3 = \sin(5x+5)$", fontsize=14)
plt.legend()
plt.axhline(1, color='gray', linestyle='--', alpha=0.3)
plt.axhline(-1, color='gray', linestyle='--', alpha=0.3)
plt.show()

print("\nNote: Smaller epsilon creates sharper internal transition layers.")

## 7. INGU: Inexact Newton-GMRES-Ultraspherical

For larger problems, the matrix-based Newton method becomes expensive because:
1. Assembling the Jacobian matrix costs $O(n^2)$
2. Solving the linear system with LU costs $O(n^3)$

The **INGU algorithm** (Qin & Xu, 2024) avoids these costs by:
1. Using **FFT-based matrix-vector products** - $O(n \log n)$ per product
2. Using **GMRES** (an iterative Krylov method) instead of direct LU
3. Using an **almost-banded preconditioner** for fast convergence

This is especially powerful for problems requiring many discretization points.

In [ ]:
# Compare matrix-based Newton vs INGU on a moderately sized problem
import time

n = 128  # discretization size

# Problem: pendulum equation u'' + 25*sin(u) = 0
def solve_with_method(method, n):
    op = ChebOp(functions=['u'], n=n, domain=[-1, 1])
    op.eqn = ['diff(u, x, 2) + 25*sin(u)']
    op.bcs = ['u(-1) - 2', 'u(1) - 2']
    
    start = time.time()
    soln, success, res = op.solve(method=method, verbose=False, adaptive=False)
    elapsed = time.time() - start
    
    return soln, success, res, elapsed

# Solve with INGU method
soln_ingu, success_i, res_i, time_i = solve_with_method('ingu', n)

print(f"INGU: time = {time_i:.3f}s, residual = {res_i:.2e}")
print(f"Solution at x=0: {soln_ingu(0):.6f}")

In [ ]:
# Plot the INGU solution
x = np.linspace(-1, 1, 200)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(x, soln_ingu(x), 'b-', lw=2)
axes[0].set_xlabel('x')
axes[0].set_ylabel('u(x)')
axes[0].set_title(r"Pendulum: $u'' + 25\sin(u) = 0$, solved with INGU")
axes[0].axhline(0, color='gray', linestyle='--', alpha=0.5)

# Verify residual
residual = soln_ingu.diff(2) + 25 * np.sin(soln_ingu)
axes[1].semilogy(x, np.abs(residual(x)) + 1e-16, 'r-', lw=2)
axes[1].set_xlabel('x')
axes[1].set_ylabel(r"|$u'' + 25\sin(u)$|")
axes[1].set_title('Residual')

plt.tight_layout()
plt.show()

## 8. INGU Scaling

Let's see how INGU scales with problem size. The key advantage of INGU is:
- FFT-based matrix-vector products: $O(n \log n)$ per GMRES iteration
- With a good preconditioner, the number of GMRES iterations $k$ is small and nearly independent of $n$
- Overall complexity: $O(n \log n \cdot k)$ per Newton step

In [ ]:
# INGU scaling test
sizes = [32, 64, 128, 256]
times_ingu = []

for n in sizes:
    print(f"n = {n}...", end=' ')
    
    _, _, _, t_i = solve_with_method('ingu', n)
    times_ingu.append(t_i)
    
    print(f"INGU: {t_i:.3f}s")

In [ ]:
# Plot INGU scaling
plt.figure(figsize=(8, 5))
plt.loglog(sizes, times_ingu, 'rs-', lw=2, markersize=8, label='INGU')

# Reference lines
n_ref = np.array(sizes)
scale = times_ingu[1] / (n_ref[1] * np.log(n_ref[1]))
plt.loglog(n_ref, scale * n_ref * np.log(n_ref), 'r--', alpha=0.5, label=r'$O(n \log n)$')

plt.xlabel('Discretization size n', fontsize=12)
plt.ylabel('Time (seconds)', fontsize=12)
plt.title('INGU Scaling: Pendulum Equation', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\nNote: INGU scales nearly linearly due to FFT-based operators.")

## 9. Summary

**Key takeaways:**

1. **Nonlinear BVPs** require iterative methods like Newton's method
2. Each Newton step solves a **linearized problem** with variable coefficients
3. The **ultraspherical spectral method** achieves spectral accuracy
4. **Matrix-based Newton** is simple but scales as $O(n^3)$
5. **INGU** uses FFT-based operators and GMRES for $O(n \log n)$ scaling
6. For large problems (n > 100-200), INGU becomes more efficient

**Classic test problems:**
- **Bratu equation**: Combustion, thermal ignition
- **Fisher equation**: Population dynamics, traveling waves  
- **Allen-Cahn**: Phase transitions, interface dynamics
- **Carrier equation**: Singular perturbations

## References

1. Qin, Y. & Xu, K. (2024). "Solving Nonlinear ODEs with the Ultraspherical Spectral Method." *IMA J. Numer. Anal.* doi:10.1093/imanum/drad099

2. Olver, S. & Townsend, A. (2013). "A Fast and Well-Conditioned Spectral Method." *SIAM Review*, 55(3), 462-489.

3. Trefethen, L. N. (2000). *Spectral Methods in MATLAB*. SIAM.